In [1]:
#Load in packages
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import contextily as cx
import warnings
import os

In [ ]:
#Load in 

In [ ]:
#Assign each census block to a police beat with for loop
cblockIndexToBeat = {}
sIndex = policebeats.sindex

for cblock_ind in durhamblockmap.index:
    cblock_geom = durhamblockmap.loc[cblock_ind]["geometry"]
    possibleBeatMatches = list(sIndex.intersection(cblock_geom.bounds))
    matched = False

    if len(possibleBeatMatches) > 0:
        possibleMatches = policebeats.iloc[possibleBeatMatches]
        intersectingAreas = possibleMatches.intersection(cblock_geom).area
        intersectingAreas /= cblock_geom.area
        police_beat_index = intersectingAreas.idxmax()
        maxIntersection = intersectingAreas.max()
    ##########
        if maxIntersection < 0.1:
            continue
    ##########
        if maxIntersection > 0:
            cblockIndexToBeat[cblock_ind] = policebeats.loc[police_beat_index]['LAWBEAT']
            matched = True

cblockDurhamBeats = ncblockmap[ncblockmap.index.isin(cblockIndexToBeat)]
cblockDurhamBeats['Beat'] = cblockDurhamBeats.index.map(lambda ii: cblockIndexToBeat[ii])

warnings.filterwarnings('ignore')

In [ ]:
#Make index GEOID20
durhamracebeat = cblockDurhamBeats.set_index('GEOID20').join(racedata.set_index('GEOID20'))
durhamracebeat.head(3)

In [ ]:
#Change the column names
durhamracebeat.columns = durhamracebeat.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
durhamracebeat.columns = durhamracebeat.columns.str.replace('[#,@,&,!]', '')
durhamracebeat.columns = durhamracebeat.columns.str.replace('[:]', '_')
durhamracebeat.head()

warnings.filterwarnings('ignore')

In [ ]:
#Find the proportion of specific races compared to the total population
durhamracebeat['propwhite']= durhamracebeat['total_population_of_one_race_white_alone'] / durhamracebeat['total_']
durhamracebeat['propblack']= durhamracebeat['total_population_of_one_race_black_or_african_american_alone'] / durhamracebeat['total_']

In [ ]:
#Plot chloropleth map with merged data for white population
fig, ax = plt.subplots(figsize = (10,10))

durhamracebeat.plot(column='propwhite',
                                legend=True,
                                cmap="viridis", ax=ax, aspect=1);

plt.title("Proportion of each Beat in Durham that is White alone\n(Does not include mixed persons)")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()

In [ ]:
#Plot chloropleth map with merged data for Black population
fig, ax = plt.subplots(figsize = (10,10))

durhamracebeat.plot(column='propblack',
                                legend=True,
                                cmap="viridis", ax=ax, aspect=1);

plt.title("Proportion of each Block in Durham that is Black or African American alone\n(Does not include mixed persons)")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()

In [ ]:
#Merging race data into the police beat and block data
beatswithrace = pd.merge(racedata, cblockDurhamBeats, on ='GEOID20')
beatswithrace.head(3)

In [ ]:
#Grouping the population by police beat
popsumbybeat = beatswithrace.groupby('Beat', as_index=False).sum()
popsumbybeat.head()

In [ ]:
#Adding beat geometry to popsumbybeat
raceineachbeat = pd.merge(popsumbybeat, policebeats, left_on='Beat', right_on='LAWBEAT')
raceineachbeat.head(3)

In [ ]:
#Setting the beat as the index
raceineachbeat = popsumbybeat.set_index('Beat').join(policebeats.set_index('LAWBEAT'))
raceineachbeat.head(3)

In [ ]:
#Seeing how many blocks there are per beat
blocksperbeat = beatswithrace.groupby( ["Beat"] ).size().to_frame(name = 'block_count').reset_index()
meanblocksperbeat = blocksperbeat[["block_count"]].mean()
meanblocksperbeat